<a href="https://colab.research.google.com/github/123nishant/Google-Colab/blob/master/Jokes_using_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 675kB 4.7MB/s 
     |████████████████████████████████| 3.8MB 18.8MB/s 
     |████████████████████████████████| 890kB 49.1MB/s 
     |████████████████████████████████| 1.1MB 50.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=519ec8066e7041eada38b668b5798cfacd97bb940fc87a96d35d79a5042d5fc5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)

import torch
import numpy as np

from transformers import GPT2Tokenizer, GPT2LMHeadModel

device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda'

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

In [0]:
# Function to select topN tokens from the probability list and then based on the selected 
# N word distribution 
# Get random token ID
def choose_from_top(probs, n=5):
  ind = np.argpartition(probs, -n)[-n:]
  top_prob = probs[ind]
  top_prob = top_prob / np.sum(top_prob) # Normalize
  choice = np.random.choice(n, 1, p=top_prob)
  token_id = ind[choice][0]
  return int(token_id)

In [0]:
def generate_some_text(input_str, text_len = 250):
  cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)

  model.eval()
  with torch.no_grad():

    for i in range(text_len):
      outputs = model(cur_ids, labels=cur_ids)
      loss, logits = outputs[:2]
      softmax_logits = torch.softmax(logits[0, -1], dim=0)
      # Taking the first only one batch and the last predicted embedding
      next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=10)
      # Randomly( from the given probability distribution) choose the next word fromtop n words
      cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
      # Add the last word
      output_list = list(cur_ids.squeeze().to('cpu').numpy())
      output_text = tokenizer.decode(output_list)
      print(output_text)

In [14]:
generate_some_text(" The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. ")

 The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. It
 The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. It is
 The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. It i

In [15]:
generate_some_text(" Artificial general intelligence is ")

 Artificial general intelligence is a
 Artificial general intelligence is a concept
 Artificial general intelligence is a concept that
 Artificial general intelligence is a concept that is
 Artificial general intelligence is a concept that is difficult
 Artificial general intelligence is a concept that is difficult to
 Artificial general intelligence is a concept that is difficult to understand
 Artificial general intelligence is a concept that is difficult to understand for
 Artificial general intelligence is a concept that is difficult to understand for the
 Artificial general intelligence is a concept that is difficult to understand for the unin
 Artificial general intelligence is a concept that is difficult to understand for the uniniti
 Artificial general intelligence is a concept that is difficult to understand for the uninitiated
 Artificial general intelligence is a concept that is difficult to understand for the uninitiated.
 Artificial general intelligence is a concept that i

In [16]:
generate_some_text(" The Godfather: \"I'm going to make him an offer he can't refuse.\" ")

 The Godfather: "I'm going to make him an offer he can't refuse." This
 The Godfather: "I'm going to make him an offer he can't refuse." This was
 The Godfather: "I'm going to make him an offer he can't refuse." This was a
 The Godfather: "I'm going to make him an offer he can't refuse." This was a direct
 The Godfather: "I'm going to make him an offer he can't refuse." This was a direct quote
 The Godfather: "I'm going to make him an offer he can't refuse." This was a direct quote from
 The Godfather: "I'm going to make him an offer he can't refuse." This was a direct quote from Don
 The Godfather: "I'm going to make him an offer he can't refuse." This was a direct quote from Don's
 The Godfather: "I'm going to make him an offer he can't refuse." This was a direct quote from Don's son
 The Godfather: "I'm going to make him an offer he can't refuse." This was a direct quote from Don's son Christopher
 The Godfather: "I'm going to make him an offer he can't refuse." This was a direct qu

# Fine-tuning on jokes

In [0]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'

if torch.cuda.is_available():
  device = 'cuda'

In [0]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

In [0]:
def choose_from_top(probs, n=5):
  ind = np.argpartition(probs, -n)[-n:]
  top_prob = probs[ind]
  top_prob = top_prob / np.sum(top_prob) # Normalize  by layer
  choice = np.random.choice(n,1,p=top_prob)
  token_id = ind[choice][0]
  return int(token_id)

In [0]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import json
import csv

class JokesDataset(Dataset):
    def __init__(self, jokes_dataset_path = ''):
        super().__init__()

        short_jokes_path = os.path.join(jokes_dataset_path, 'shortjokes.csv')

        self.joke_list = []
        self.end_of_text_token = "<|endoftext|>"
        
        with open(short_jokes_path) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            
            x = 0
            for row in csv_reader:
                joke_str = f"JOKE:{row[1]}{self.end_of_text_token}"
                self.joke_list.append(joke_str)
        
    def __len__(self):
        return len(self.joke_list)

    def __getitem__(self, item):
        return self.joke_list[item]

In [0]:
dataset = JokesDataset()
joke_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [0]:
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 3e-5
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 400
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup as WarmupLinearSchedule

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [28]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS, t_total = -1)
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_jokes_tens = None
models_folder = "trained_models"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)
    
    for idx,joke in enumerate(joke_loader):
        
        #################### "Fit as many joke sequences into MAX_SEQ_LEN sequence as possible" logic start ####
        joke_tens = torch.tensor(tokenizer.encode(joke[0])).unsqueeze(0).to(device)
        #Skip sample from dataset if it is longer than MAX_SEQ_LEN
        if joke_tens.size()[1] > MAX_SEQ_LEN:
            continue
        
        #The first joke sequence in the sequence
        if not torch.is_tensor(tmp_jokes_tens):
            tmp_jokes_tens = joke_tens
            continue
        else:
            #The next joke does not fit in so we process the sequence and leave the last joke 
            #as the start for next sequence 
            if tmp_jokes_tens.size()[1] + joke_tens.size()[1] > MAX_SEQ_LEN:
                work_jokes_tens = tmp_jokes_tens
                tmp_jokes_tens = joke_tens
            else:
                #Add the joke to sequence, continue and try to add more
                tmp_jokes_tens = torch.cat([tmp_jokes_tens, joke_tens[:,1:]], dim=1)
                continue
        ################## Sequence ready, process it trough the model ##################
            
        outputs = model(work_jokes_tens, labels=work_jokes_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 100:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0
    
    # Store the model after each epoch to compare the performance of them
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_joker_{epoch}.pt"))

TypeError: ignored

In [0]:
MODEL_EPOCH = 4

models_folder = "trained_models"

model_path = os.path.join(models_folder, f"gpt2_medium_joker_{MODEL_EPOCH}.pt")
model.load_state_dict(torch.load(model_path))

jokes_output_file_path = f'generated_{MODEL_EPOCH}.jokes'

model.eval()
if os.path.exists(jokes_output_file_path):
    os.remove(jokes_output_file_path)
    
joke_num = 0
with torch.no_grad():
   
        for joke_idx in range(1000):
        
            joke_finished = False

            cur_ids = torch.tensor(tokenizer.encode("JOKE:")).unsqueeze(0).to(device)

            for i in range(100):
                outputs = model(cur_ids, labels=cur_ids)
                loss, logits = outputs[:2]
                softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
                if i < 3:
                    n = 20
                else:
                    n = 3
                next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the topN probability distribution) select the next word
                cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

                if next_token_id in tokenizer.encode('<|endoftext|>'):
                    joke_finished = True
                    break

            
            if joke_finished:
                
                joke_num = joke_num + 1
                
                output_list = list(cur_ids.squeeze().to('cpu').numpy())
                output_text = tokenizer.decode(output_list)

                with open(jokes_output_file_path, 'a') as f:
                    f.write(f"{output_text} \n\n")